In [1]:
""" script for training the MSG-GAN on given dataset """

# Set to True if using in SageMaker
USE_SAGEMAKER = False

import argparse

import os
import numpy as np
import torch as th
from torch.backends import cudnn

# sagemaker_containers required to access SageMaker environment (SM_CHANNEL_TRAINING, etc.)
# See https://github.com/aws/sagemaker-containers
if USE_SAGEMAKER:
    import sagemaker_containers

# define the device for the training script
device = th.device("cuda" if th.cuda.is_available() else "cpu")

# enable fast training
cudnn.benchmark = True

# set seed = 3
th.manual_seed(seed=3)


gen_file = 'C:\\Users\\mati\\Desktop\\Workplace\\jupyter_DD\\MSGgan\\BMSG-GAN-master\\sourcecode\\models\\exp_2\\GAN_GEN_10.pth'
gen_optim_file ='C:\\Users\\mati\\Desktop\\Workplace\\jupyter_DD\\MSGgan\\BMSG-GAN-master\\sourcecode\\models\\exp_2\\GAN_GEN_OPTIM_10.pth'
gen_shadow_file = 'C:\\Users\\mati\\Desktop\\Workplace\\jupyter_DD\\MSGgan\\BMSG-GAN-master\\sourcecode\\models\\exp_2\\GAN_GEN_SHADOW_10.pth'

disc_file = 'C:\\Users\\mati\\Desktop\\Workplace\\jupyter_DD\\MSGgan\\BMSG-GAN-master\\sourcecode\\models\\exp_2\\GAN_DIS_10.pth'
disc_optim_file = 'C:\\Users\\mati\\Desktop\\Workplace\\jupyter_DD\\MSGgan\\BMSG-GAN-master\\sourcecode\\models\\exp_2\\GAN_DIS_OPTIM_10.pth'
gen_file = None
gen_optim_file = None
gen_shadow_file = None
disc_file = None
disc_optim_file = None

def parse_arguments():
    """
    command line arguments parser
    :return: args => parsed command line arguments
    """
    parser = argparse.ArgumentParser()

    parser.add_argument("--generator_file", action="store", type=str,
                        default=gen_file,
                        help="pretrained weights file for generator")

    parser.add_argument("--generator_optim_file", action="store", type=str,
                        default=gen_optim_file,
                        help="saved state for generator optimizer")

    parser.add_argument("--shadow_generator_file", action="store", type=str,
                        default=gen_shadow_file,
                        help="pretrained weights file for the shadow generator")

    parser.add_argument("--discriminator_file", action="store", type=str,
                        default=disc_file,
                        help="pretrained_weights file for discriminator")

    parser.add_argument("--discriminator_optim_file", action="store", type=str,
                        default=disc_optim_file,
                        help="saved state for discriminator optimizer")

    parser.add_argument("--images_dir", action="store", type=str,
                        # default="../data/celeba",
                        default='C:\\Users\\mati\\jupyter\\celeb100k\\images',
                        help="path for the images directory")

    parser.add_argument("--folder_distributed", action="store", type=bool,
                        default=False,
                        help="whether the images directory contains folders or not")

    parser.add_argument("--flip_augment", action="store", type=bool,
                        default=True,
                        help="whether to randomly mirror the images during training")

    parser.add_argument("--sample_dir", action="store", type=str,
                        default="samples/exp_128/",
                        #default=os.environ['SM_MODEL_DIR'],
                        help="path for the generated samples directory")

    parser.add_argument("--model_dir", action="store", type=str,
                        default="models/exp_128/",
                        #default=os.environ['SM_MODEL_DIR'],
                        help="path for saved models directory")

    parser.add_argument("--loss_function", action="store", type=str,
                        default="relativistic-hinge",
                        help="loss function to be used: " +
                             "standard-gan, wgan-gp, lsgan," +
                             "lsgan-sigmoid," +
                             "hinge, relativistic-hinge")

    parser.add_argument("--depth", action="store", type=int,
                        default=6, # 64x64
                        help="Depth of the GAN")

    parser.add_argument("--latent_size", action="store", type=int,
                        default=512,
                        help="latent size for the generator")

    parser.add_argument("--batch_size", action="store", type=int,
                        default=12,
                        help="batch_size for training")

    parser.add_argument("--start", action="store", type=int,
                        default=10,
                        help="starting epoch number")

    parser.add_argument("--num_epochs", action="store", type=int,
                        default=300,
                        help="number of epochs for training")

    parser.add_argument("--feedback_factor", action="store", type=int,
                        default=100,
                        help="number of logs to generate per epoch")

    parser.add_argument("--num_samples", action="store", type=int,
                        default=36,
                        help="number of samples to generate for creating the grid" +
                             " should be a square number preferably")

    parser.add_argument("--checkpoint_factor", action="store", type=int,
                        default=1,
                        help="save model per n epochs")

    parser.add_argument("--g_lr", action="store", type=float,
                        default=0.003,
                        help="learning rate for generator")

    parser.add_argument("--d_lr", action="store", type=float,
                        default=0.003,
                        help="learning rate for discriminator")

    parser.add_argument("--adam_beta1", action="store", type=float,
                        default=0,
                        help="value of beta_1 for adam optimizer")

    parser.add_argument("--adam_beta2", action="store", type=float,
                        default=0.99,
                        help="value of beta_2 for adam optimizer")

    parser.add_argument("--use_eql", action="store", type=bool,
                        default=True,
                        help="Whether to use equalized learning rate or not")

    parser.add_argument("--use_ema", action="store", type=bool,
                        default=True,
                        help="Whether to use exponential moving averages or not")

    parser.add_argument("--ema_decay", action="store", type=float,
                        default=0.999,
                        help="decay value for the ema")

    parser.add_argument("--data_percentage", action="store", type=float,
                        default=100,
                        help="percentage of data to use")

    parser.add_argument("--num_workers", action="store", type=int,
                        default=3,
                        help="number of parallel workers for reading files")

    args = parser.parse_args(args=[])
    print('args={}'.format(args))

    return args


C:\Users\mati\anaconda3\envs\GPU-pytorch-2022-12\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
args = parse_arguments()

args=Namespace(generator_file=None, generator_optim_file=None, shadow_generator_file=None, discriminator_file=None, discriminator_optim_file=None, images_dir='C:\\Users\\mati\\jupyter\\celeb100k\\images', folder_distributed=False, flip_augment=True, sample_dir='samples/exp_128/', model_dir='models/exp_128/', loss_function='relativistic-hinge', depth=6, latent_size=512, batch_size=12, start=10, num_epochs=300, feedback_factor=100, num_samples=36, checkpoint_factor=1, g_lr=0.003, d_lr=0.003, adam_beta1=0, adam_beta2=0.99, use_eql=True, use_ema=True, ema_decay=0.999, data_percentage=100, num_workers=3)


In [3]:
from MSG_GAN.GAN import MSG_GAN
from data_processing.DataLoader import FlatDirectoryImageDataset, get_transform, get_data_loader, FoldersDistributedDataset
from MSG_GAN import Losses as lses

## Main

In [4]:
# create a data source:
data_source = FlatDirectoryImageDataset if not args.folder_distributed \
    else FoldersDistributedDataset

dataset = data_source(
    args.images_dir,
    transform=get_transform((int(np.power(2, args.depth + 1)),
                             int(np.power(2, args.depth + 1))),
                            flip_horizontal=args.flip_augment))

data = get_data_loader(dataset, args.batch_size, args.num_workers)
print("Total number of images in the dataset:", len(dataset))

# create a gan from these
msg_gan = MSG_GAN(depth=args.depth,
                  latent_size=args.latent_size,
                  use_eql=args.use_eql,
                  use_ema=args.use_ema,
                  ema_decay=args.ema_decay,
                  device=device)

if args.generator_file is not None:
    # load the weights into generator
    print("loading generator_weights from:", args.generator_file)
    msg_gan.gen.load_state_dict(th.load(args.generator_file))

print("Generator Configuration: ")
print(msg_gan.gen)

if args.shadow_generator_file is not None:
    # load the weights into generator
    print("loading shadow_generator_weights from:",
          args.shadow_generator_file)
    msg_gan.gen_shadow.load_state_dict(
        th.load(args.shadow_generator_file))

if args.discriminator_file is not None:
    # load the weights into discriminator
    print("loading discriminator_weights from:", args.discriminator_file)
    msg_gan.dis.load_state_dict(th.load(args.discriminator_file))

print("Discriminator Configuration: ")
print(msg_gan.dis)

# create optimizer for generator:
gen_optim = th.optim.Adam(msg_gan.gen.parameters(), args.g_lr,
                          [args.adam_beta1, args.adam_beta2])

dis_optim = th.optim.Adam(msg_gan.dis.parameters(), args.d_lr,
                          [args.adam_beta1, args.adam_beta2])

if args.generator_optim_file is not None:
    print("loading gen_optim_state from:", args.generator_optim_file)
    gen_optim.load_state_dict(th.load(args.generator_optim_file))

if args.discriminator_optim_file is not None:
    print("loading dis_optim_state from:", args.discriminator_optim_file)
    dis_optim.load_state_dict(th.load(args.discriminator_optim_file))

loss_name = args.loss_function.lower()

if loss_name == "hinge":
    loss = lses.HingeGAN
elif loss_name == "relativistic-hinge":
    loss = lses.RelativisticAverageHingeGAN
elif loss_name == "standard-gan":
    loss = lses.StandardGAN
elif loss_name == "lsgan":
    loss = lses.LSGAN
elif loss_name == "lsgan-sigmoid":
    loss = lses.LSGAN_SIGMOID
elif loss_name == "wgan-gp":
    loss = lses.WGAN_GP
else:
    raise Exception("Unknown loss function requested")




Total number of images in the dataset: 100000
Generator Configuration: 
DataParallel(
  (module): Generator(
    (layers): ModuleList(
      (0): GenInitialBlock(
        (conv_1): _equalized_deconv2d(512, 512, 4, 4)
        (conv_2): _equalized_conv2d(512, 512, 3, 3)
        (pixNorm): PixelwiseNorm()
        (lrelu): LeakyReLU(negative_slope=0.2)
      )
      (1): GenGeneralConvBlock(
        (conv_1): _equalized_conv2d(512, 512, 3, 3)
        (conv_2): _equalized_conv2d(512, 512, 3, 3)
        (pixNorm): PixelwiseNorm()
        (lrelu): LeakyReLU(negative_slope=0.2)
      )
      (2): GenGeneralConvBlock(
        (conv_1): _equalized_conv2d(512, 512, 3, 3)
        (conv_2): _equalized_conv2d(512, 512, 3, 3)
        (pixNorm): PixelwiseNorm()
        (lrelu): LeakyReLU(negative_slope=0.2)
      )
      (3): GenGeneralConvBlock(
        (conv_1): _equalized_conv2d(512, 512, 3, 3)
        (conv_2): _equalized_conv2d(512, 512, 3, 3)
        (pixNorm): PixelwiseNorm()
        (lrelu): L

In [ ]:
# train the GAN
msg_gan.train(
    data,
    gen_optim,
    dis_optim,
    loss_fn=loss(msg_gan.dis),
    num_epochs=args.num_epochs,
    checkpoint_factor=args.checkpoint_factor,
    data_percentage=args.data_percentage,
    feedback_factor=args.feedback_factor,
    num_samples=args.num_samples,
    sample_dir=args.sample_dir,
    save_dir=args.model_dir,
    log_dir=args.model_dir,
    start=0
)


Starting the training process ... 

Epoch: 0
Limit =  8334
Elapsed [0:00:06.837459] batch: 1  d_loss: 3.371750  g_loss: 26.254230
Elapsed [0:00:19.330029] batch: 20  d_loss: 0.000000  g_loss: 26.980717
Elapsed [0:00:30.653640] batch: 40  d_loss: 0.000000  g_loss: 16.957531
Elapsed [0:00:42.541270] batch: 60  d_loss: 0.000000  g_loss: 24.788212
Elapsed [0:00:54.144880] batch: 80  d_loss: 0.000000  g_loss: 18.636009
Elapsed [0:01:06.352639] batch: 100  d_loss: 0.000000  g_loss: 21.954113
Elapsed [0:01:17.719978] batch: 120  d_loss: 0.000000  g_loss: 14.441682
Elapsed [0:01:29.081844] batch: 140  d_loss: 0.000000  g_loss: 11.010797
Elapsed [0:01:40.381919] batch: 160  d_loss: 0.000000  g_loss: 29.223938
Elapsed [0:01:51.698962] batch: 180  d_loss: 0.035453  g_loss: 9.453754
Elapsed [0:02:03.555770] batch: 200  d_loss: 0.281761  g_loss: 7.407709
Elapsed [0:02:15.553251] batch: 220  d_loss: 0.520699  g_loss: 11.430902
Elapsed [0:02:27.240871] batch: 240  d_loss: 0.000000  g_loss: 8.604771
E

Elapsed [0:21:22.201986] batch: 2220  d_loss: 0.001948  g_loss: 13.984568
Elapsed [0:21:33.407490] batch: 2240  d_loss: 0.059811  g_loss: 14.665527
Elapsed [0:21:44.633115] batch: 2260  d_loss: 0.000426  g_loss: 10.401645
Elapsed [0:21:55.813476] batch: 2280  d_loss: 0.015977  g_loss: 12.988638
Elapsed [0:22:06.981201] batch: 2300  d_loss: 0.000000  g_loss: 8.680136
Elapsed [0:22:18.175802] batch: 2320  d_loss: 1.140885  g_loss: 33.638977
Elapsed [0:22:29.344612] batch: 2340  d_loss: 0.000000  g_loss: 15.494646
Elapsed [0:22:40.513347] batch: 2360  d_loss: 0.000000  g_loss: 10.264637
Elapsed [0:22:51.712274] batch: 2380  d_loss: 0.423143  g_loss: 13.547070
Elapsed [0:23:02.886517] batch: 2400  d_loss: 0.000000  g_loss: 15.007332
Elapsed [0:23:14.102039] batch: 2420  d_loss: 0.008002  g_loss: 12.843778
Elapsed [0:23:25.292007] batch: 2440  d_loss: 0.000000  g_loss: 21.372318
Elapsed [0:23:36.529626] batch: 2460  d_loss: 0.000000  g_loss: 12.730900
Elapsed [0:23:47.770939] batch: 2480  d

Elapsed [0:42:15.524570] batch: 4440  d_loss: 0.024449  g_loss: 21.841326
Elapsed [0:42:27.479108] batch: 4460  d_loss: 0.000000  g_loss: 18.785225
Elapsed [0:42:39.491590] batch: 4480  d_loss: 1.031954  g_loss: 27.379967
Elapsed [0:42:51.513148] batch: 4500  d_loss: 0.164933  g_loss: 15.341352
Elapsed [0:43:03.517414] batch: 4520  d_loss: 0.502148  g_loss: 17.996250
Elapsed [0:43:15.503471] batch: 4540  d_loss: 0.000000  g_loss: 11.420979
Elapsed [0:43:27.583339] batch: 4560  d_loss: 0.000000  g_loss: 22.028381
Elapsed [0:43:39.424368] batch: 4580  d_loss: 0.142410  g_loss: 13.519290
Elapsed [0:43:50.931393] batch: 4600  d_loss: 0.000000  g_loss: 15.114714
Elapsed [0:44:02.353739] batch: 4620  d_loss: 0.662736  g_loss: 9.782850
Elapsed [0:44:13.799652] batch: 4640  d_loss: 0.796026  g_loss: 10.062855
Elapsed [0:44:25.209791] batch: 4660  d_loss: 0.064558  g_loss: 13.874304
Elapsed [0:44:36.649022] batch: 4680  d_loss: 0.082802  g_loss: 12.762089
Elapsed [0:44:48.109160] batch: 4700  d

Elapsed [1:03:23.042809] batch: 6660  d_loss: 0.000000  g_loss: 13.923736
Elapsed [1:03:34.821167] batch: 6680  d_loss: 0.000000  g_loss: 22.479343
Elapsed [1:03:46.171976] batch: 6700  d_loss: 0.000000  g_loss: 18.019909
Elapsed [1:03:58.263702] batch: 6720  d_loss: 0.000000  g_loss: 12.947151
Elapsed [1:04:10.015302] batch: 6740  d_loss: 0.000000  g_loss: 51.961994
Elapsed [1:04:21.464197] batch: 6760  d_loss: 0.000000  g_loss: 14.398479
Elapsed [1:04:32.872888] batch: 6780  d_loss: 0.000000  g_loss: 21.018734
Elapsed [1:04:44.303692] batch: 6800  d_loss: 0.000000  g_loss: 21.162279
Elapsed [1:04:55.675625] batch: 6820  d_loss: 0.000000  g_loss: 52.514263
Elapsed [1:05:07.093222] batch: 6840  d_loss: 0.105440  g_loss: 27.769577
Elapsed [1:05:18.475162] batch: 6860  d_loss: 0.719031  g_loss: 30.201357
Elapsed [1:05:29.878243] batch: 6880  d_loss: 0.000000  g_loss: 10.313354
Elapsed [1:05:41.264632] batch: 6900  d_loss: 0.866691  g_loss: 17.262138
Elapsed [1:05:52.656624] batch: 6920  

Elapsed [1:25:02.512102] batch: 540  d_loss: 0.000000  g_loss: 21.196247
Elapsed [1:25:15.199254] batch: 560  d_loss: 0.524914  g_loss: 43.006554
Elapsed [1:25:28.008361] batch: 580  d_loss: 0.091659  g_loss: 10.781559
Elapsed [1:25:40.709080] batch: 600  d_loss: 0.012972  g_loss: 17.414204
Elapsed [1:25:53.471586] batch: 620  d_loss: 8.118458  g_loss: 25.616438
Elapsed [1:26:06.184070] batch: 640  d_loss: 5.718196  g_loss: 44.529758
Elapsed [1:26:18.948697] batch: 660  d_loss: 0.000000  g_loss: 10.515673
Elapsed [1:26:31.633316] batch: 680  d_loss: 0.000000  g_loss: 8.039688
Elapsed [1:26:44.329025] batch: 700  d_loss: 0.000000  g_loss: 13.484129
Elapsed [1:26:57.060155] batch: 720  d_loss: 1.204284  g_loss: 47.076378
Elapsed [1:27:09.885402] batch: 740  d_loss: 0.273585  g_loss: 11.907219
Elapsed [1:27:22.555724] batch: 760  d_loss: 0.180504  g_loss: 12.372285
Elapsed [1:27:35.220169] batch: 780  d_loss: 0.097023  g_loss: 15.842222
Elapsed [1:27:47.200602] batch: 800  d_loss: 0.00000

...

Elapsed [1 day, 9:45:34.702888] batch: 7080  d_loss: 0.000000  g_loss: 18.183310
Elapsed [1 day, 9:45:46.784624] batch: 7100  d_loss: 0.000000  g_loss: 17.706427
Elapsed [1 day, 9:45:59.158175] batch: 7120  d_loss: 0.000000  g_loss: 19.289881
Elapsed [1 day, 9:46:11.458081] batch: 7140  d_loss: 0.000000  g_loss: 12.761377
Elapsed [1 day, 9:46:23.458270] batch: 7160  d_loss: 0.000000  g_loss: 26.578032
Elapsed [1 day, 9:46:35.550099] batch: 7180  d_loss: 0.000000  g_loss: 16.601831
Elapsed [1 day, 9:46:47.566255] batch: 7200  d_loss: 0.000000  g_loss: 18.893799
Elapsed [1 day, 9:46:59.510525] batch: 7220  d_loss: 0.000000  g_loss: 19.928928
Elapsed [1 day, 9:47:11.852178] batch: 7240  d_loss: 1.091671  g_loss: 19.695225
Elapsed [1 day, 9:47:23.881127] batch: 7260  d_loss: 0.046136  g_loss: 12.833920
Elapsed [1 day, 9:47:36.078302] batch: 7280  d_loss: 0.000000  g_loss: 14.543356
Elapsed [1 day, 9:47:48.061415] batch: 7300  d_loss: 0.000000  g_loss: 10.831717
Elapsed [1 day, 9:48:00.3415

Elapsed [1 day, 10:06:14.708006] batch: 760  d_loss: 0.000000  g_loss: 15.830379
Elapsed [1 day, 10:06:27.250945] batch: 780  d_loss: 0.000000  g_loss: 16.333893
Elapsed [1 day, 10:06:39.597537] batch: 800  d_loss: 0.175497  g_loss: 10.722461
Elapsed [1 day, 10:06:51.880140] batch: 820  d_loss: 0.000000  g_loss: 8.874415
Elapsed [1 day, 10:07:03.976334] batch: 840  d_loss: 0.000000  g_loss: 9.501389
Elapsed [1 day, 10:07:16.170746] batch: 860  d_loss: 0.000000  g_loss: 14.638658
Elapsed [1 day, 10:07:28.686174] batch: 880  d_loss: 0.000000  g_loss: 9.487933
Elapsed [1 day, 10:07:41.382684] batch: 900  d_loss: 0.000000  g_loss: 12.908781
Elapsed [1 day, 10:07:53.750455] batch: 920  d_loss: 0.000000  g_loss: 18.004316
Elapsed [1 day, 10:08:06.325740] batch: 940  d_loss: 0.000000  g_loss: 17.930534
Elapsed [1 day, 10:08:18.884763] batch: 960  d_loss: 0.000000  g_loss: 12.307977
Elapsed [1 day, 10:08:31.187413] batch: 980  d_loss: 0.000000  g_loss: 16.496990
Elapsed [1 day, 10:08:43.462051

Elapsed [1 day, 10:27:00.010736] batch: 2780  d_loss: 0.000000  g_loss: 17.868200
Elapsed [1 day, 10:27:12.543314] batch: 2800  d_loss: 0.000000  g_loss: 19.753399
Elapsed [1 day, 10:27:24.826946] batch: 2820  d_loss: 0.000000  g_loss: 22.421398
Elapsed [1 day, 10:27:37.127284] batch: 2840  d_loss: 0.000000  g_loss: 12.138367
Elapsed [1 day, 10:27:49.240618] batch: 2860  d_loss: 0.000000  g_loss: 8.337128
Elapsed [1 day, 10:28:01.677344] batch: 2880  d_loss: 0.000000  g_loss: 10.707035
Elapsed [1 day, 10:28:14.190239] batch: 2900  d_loss: 0.000000  g_loss: 24.517834
Elapsed [1 day, 10:28:26.455846] batch: 2920  d_loss: 0.000000  g_loss: 20.057503
Elapsed [1 day, 10:28:38.558220] batch: 2940  d_loss: 0.000000  g_loss: 19.988899
Elapsed [1 day, 10:28:50.888545] batch: 2960  d_loss: 0.322011  g_loss: 22.486198
Elapsed [1 day, 10:29:03.183936] batch: 2980  d_loss: 0.000000  g_loss: 19.751492
Elapsed [1 day, 10:29:15.288990] batch: 3000  d_loss: 0.000000  g_loss: 33.404305
Elapsed [1 day, 1

Elapsed [1 day, 10:47:29.982832] batch: 4780  d_loss: 0.000000  g_loss: 10.344496
Elapsed [1 day, 10:47:42.336967] batch: 4800  d_loss: 0.000000  g_loss: 15.907412
Elapsed [1 day, 10:47:54.614828] batch: 4820  d_loss: 0.000000  g_loss: 18.116238
Elapsed [1 day, 10:48:07.077931] batch: 4840  d_loss: 0.000000  g_loss: 18.766628
Elapsed [1 day, 10:48:19.157891] batch: 4860  d_loss: 0.000000  g_loss: 14.770135
Elapsed [1 day, 10:48:31.355234] batch: 4880  d_loss: 0.000000  g_loss: 19.224152
Elapsed [1 day, 10:48:43.521862] batch: 4900  d_loss: 0.000000  g_loss: 12.571561
Elapsed [1 day, 10:48:56.007392] batch: 4920  d_loss: 0.000000  g_loss: 7.653864
Elapsed [1 day, 10:49:08.358857] batch: 4940  d_loss: 0.000000  g_loss: 13.712704
Elapsed [1 day, 10:49:20.662381] batch: 4960  d_loss: 0.000000  g_loss: 14.291148
Elapsed [1 day, 10:49:33.236488] batch: 4980  d_loss: 0.000000  g_loss: 16.781094
Elapsed [1 day, 10:49:45.480177] batch: 5000  d_loss: 0.000000  g_loss: 23.389231
Elapsed [1 day, 1

Elapsed [1 day, 11:08:20.717573] batch: 6780  d_loss: 0.000000  g_loss: 22.672245
Elapsed [1 day, 11:08:35.384294] batch: 6800  d_loss: 0.000000  g_loss: 10.430663
Elapsed [1 day, 11:08:50.804760] batch: 6820  d_loss: 0.000000  g_loss: 10.716521
Elapsed [1 day, 11:09:06.239729] batch: 6840  d_loss: 0.000000  g_loss: 15.214928
Elapsed [1 day, 11:09:18.513465] batch: 6860  d_loss: 0.000000  g_loss: 12.474653
Elapsed [1 day, 11:09:30.753373] batch: 6880  d_loss: 0.000000  g_loss: 8.501728
Elapsed [1 day, 11:09:42.945676] batch: 6900  d_loss: 0.000000  g_loss: 9.516390
Elapsed [1 day, 11:09:55.938357] batch: 6920  d_loss: 0.000000  g_loss: 18.951267
Elapsed [1 day, 11:10:08.752332] batch: 6940  d_loss: 2.162451  g_loss: 37.239128
Elapsed [1 day, 11:10:21.500692] batch: 6960  d_loss: 0.000000  g_loss: 16.152641
Elapsed [1 day, 11:10:34.466195] batch: 6980  d_loss: 0.000000  g_loss: 17.177498
Elapsed [1 day, 11:10:47.064563] batch: 7000  d_loss: 0.000000  g_loss: 25.005814
Elapsed [1 day, 11